In [1]:
pwd

'/home/jovyan/work/src/generated'

Files were genereted by PubChem routines on 2023-12-08 23:47. 
(I read this on the FTP site)

In [2]:
%%capture
%%bash

cd pubchem/

wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000001.ttl.gz
wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000002.ttl.gz
wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000003.ttl.gz
wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000004.ttl.gz
wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000005.ttl.gz
wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000006.ttl.gz
wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000007.ttl.gz
wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000008.ttl.gz
wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000009.ttl.gz
wget https://ftp.ncbi.nlm.nih.gov/pubchem/RDF/inchikey/pc_inchikey2compound_000010.ttl.gz

In [3]:
%%bash

gzip -d pubchem/*.ttl.gz

In [1]:
import glob


inchikey_to_pubchem = {}


for file_path in glob.glob('pubchem/*.ttl'): 
    print(f"Parsing {file_path}...", flush=True)
    
    
    line_count = sum(1 for line in open(file_path, "r"))
    current_complete = None
    with open(file_path, "r") as file:
        
        for i, line in enumerate(file):
            line = line.strip().rstrip()
            if line.startswith('@'): continue  # it's a comment.
            fields = line.split('\t')
            
            if len(fields) == 3: 
                inchikey = fields[0].replace('inchikey:', '')
                inchikey_to_pubchem[inchikey] = set()
                pubchem = fields[2].split(' ', 1)[0]
                pubchem = pubchem.replace('compound:', '')
                inchikey_to_pubchem[inchikey].add(pubchem)
            
            if len(fields) == 1:
                pubchem = fields[0].split(' ', 1)[0]
                pubchem = pubchem.replace('compound:', '')
                inchikey_to_pubchem[inchikey].add(pubchem)
                
                
            complete = int(i/line_count*100)
            if complete % 10 == 0 and current_complete != complete:
                current_complete = complete
                print(f'{current_complete}%')
                
            
                

Parsing pubchem/pc_inchikey2compound_000004.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Parsing pubchem/pc_inchikey2compound_000001.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Parsing pubchem/pc_inchikey2compound_000010.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Parsing pubchem/pc_inchikey2compound_000008.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Parsing pubchem/pc_inchikey2compound_000006.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Parsing pubchem/pc_inchikey2compound_000007.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Parsing pubchem/pc_inchikey2compound_000009.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Parsing pubchem/pc_inchikey2compound_000002.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Parsing pubchem/pc_inchikey2compound_000005.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
Parsing pubchem/pc_inchikey2compound_000003.ttl...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%


In [2]:
import pickle
with open(f'pubchem_dicts/inchikey_to_pubchem.pickle', 'wb') as handle:
    pickle.dump(inchikey_to_pubchem, handle)

In [3]:
%pwd


'/home/jovyan/work/src/generated'

In [16]:
import copy
import pickle


def extend_with_pubchem(something_to_others, inchikey_to_pubchem):

    something_to_others_basename = something_to_others
    
    # read something_to_others
    with open(f'mnx_dicts_M/{something_to_others}.pickle', 'rb') as handler:
        something_to_others = pickle.load(handler)


    something_to_others_pubchem = copy.deepcopy(something_to_others)
    for something_id in something_to_others.keys():  # something_id = bigg or seed ID.
        something_to_others_pubchem[something_id]['pubchem.compound'] = set()

        for inchikey in something_to_others[something_id]['inchikey']:
            try:  # search this inchikey in pubchem
                key = inchikey.replace('InChIKey=', '')
                pubchem_ids = inchikey_to_pubchem[key]
                for pubchem_id in pubchem_ids:
                    something_to_others_pubchem[something_id]['pubchem.compound'].add(pubchem_id)
            except:  # this inchikey is not in pubchem  (eg WQFXOEJTASQGNX-XAXPYQTGSA-N)
                continue
                
                
    # save sopmething to others extended with pubchem
    with open(f'pubchem_dicts/{something_to_others_basename}.pickle', 'wb') as handle:
        pickle.dump(something_to_others_pubchem, handle)
    
    
    return something_to_others_pubchem
    

In [17]:
bigg_to_others =           extend_with_pubchem('bigg_to_others', inchikey_to_pubchem)
bigg_to_others_extended =  extend_with_pubchem('bigg_to_others_extended', inchikey_to_pubchem)
seed_to_others =           extend_with_pubchem('seed_to_others', inchikey_to_pubchem)
seed_to_others_extended =  extend_with_pubchem('seed_to_others_extended', inchikey_to_pubchem)

## testing area

In [18]:
seed_to_others['cpd00027']

{'kegg.compound': {'C00031'},
 'kegg.drug': {'D00009'},
 'kegg.glycan': set(),
 'biocyc': {'Glucopyranose',
  'Hederagenin-Monoglucosides',
  'Soyasapogenol-B-Monoglucosides',
  'Soyasapogenol-E-Monoglucosides'},
 'hmdb': {'HMDB0000122',
  'HMDB0000516',
  'HMDB0003340',
  'HMDB0006564',
  'HMDB00122',
  'HMDB00516',
  'HMDB0062170',
  'HMDB03340',
  'HMDB06564',
  'HMDB62170'},
 'bigg.metabolite': {'glc__D'},
 'seed.compound': {'cpd00027'},
 'chebi': {'4167'},
 'sabiork': {'1406', '1407'},
 'lipidmaps': set(),
 'envipath': set(),
 'reactome': set(),
 'rhea_g': set(),
 'rhea_p': set(),
 'swisslipids': set(),
 'inchi': {'InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2/t2-,3-,4+,5-,6?/m1/s1'},
 'inchikey': {'InChIKey=WQZGKKKJIJFFOK-GASJEMHNSA-N'},
 'smiles': {'OC[C@H]1OC(O)[C@H](O)[C@@H](O)[C@@H]1O'},
 'metanetx.chemical': {'MNXM1137670'},
 'pubchem.compound': {'CID5793'}}

In [19]:
seed_to_others_extended['cpd00027']

{'kegg.compound': {'C00031', 'C00221', 'C00267'},
 'kegg.drug': {'D00009'},
 'kegg.glycan': set(),
 'biocyc': {'ALPHA-GLUCOSE',
  'GLC',
  'Glucopyranose',
  'Hederagenin-Monoglucosides',
  'Soyasapogenol-B-Monoglucosides',
  'Soyasapogenol-E-Monoglucosides'},
 'hmdb': {'HMDB0000122',
  'HMDB0000516',
  'HMDB0003340',
  'HMDB0003345',
  'HMDB0006564',
  'HMDB00122',
  'HMDB00516',
  'HMDB0061922',
  'HMDB0062170',
  'HMDB03340',
  'HMDB03345',
  'HMDB06564',
  'HMDB61922',
  'HMDB62170'},
 'bigg.metabolite': {'Glc_aD', 'glc_D_B', 'glc__D', 'glc__aD', 'glc__bD'},
 'seed.compound': {'cpd00027', 'cpd00190', 'cpd19001'},
 'chebi': {'10242',
  '10397',
  '12318',
  '12373',
  '15903',
  '17925',
  '22386',
  '22795',
  '40557',
  '41140',
  '4167',
  '42802',
  '88300'},
 'sabiork': {'1364', '1378', '1406', '1407'},
 'lipidmaps': set(),
 'envipath': {'32de3cf4-e3e6-4168-956e-32fa5ddb0ce1/compound/bee44ace-e680-4580-8d73-374e54ae70f8',
  '650babc9-9d68-4b73-9332-11972ca26f7b/compound/16de786

In [20]:
bigg_to_others['glc__D']

{'kegg.compound': {'C00031'},
 'kegg.drug': {'D00009'},
 'kegg.glycan': set(),
 'biocyc': {'Glucopyranose',
  'Hederagenin-Monoglucosides',
  'Soyasapogenol-B-Monoglucosides',
  'Soyasapogenol-E-Monoglucosides'},
 'hmdb': {'HMDB0000122',
  'HMDB0000516',
  'HMDB0003340',
  'HMDB0006564',
  'HMDB00122',
  'HMDB00516',
  'HMDB0062170',
  'HMDB03340',
  'HMDB06564',
  'HMDB62170'},
 'bigg.metabolite': {'glc__D'},
 'seed.compound': {'cpd00027'},
 'chebi': {'4167'},
 'sabiork': {'1406', '1407'},
 'lipidmaps': set(),
 'envipath': set(),
 'reactome': set(),
 'rhea_g': set(),
 'rhea_p': set(),
 'swisslipids': set(),
 'inchi': {'InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2/t2-,3-,4+,5-,6?/m1/s1'},
 'inchikey': {'InChIKey=WQZGKKKJIJFFOK-GASJEMHNSA-N'},
 'smiles': {'OC[C@H]1OC(O)[C@H](O)[C@@H](O)[C@@H]1O'},
 'metanetx.chemical': {'MNXM1137670'},
 'pubchem.compound': {'CID5793'}}

In [21]:
bigg_to_others_extended['glc__D']

{'kegg.compound': {'C00031', 'C00221', 'C00267'},
 'kegg.drug': {'D00009'},
 'kegg.glycan': set(),
 'biocyc': {'ALPHA-GLUCOSE',
  'GLC',
  'Glucopyranose',
  'Hederagenin-Monoglucosides',
  'Soyasapogenol-B-Monoglucosides',
  'Soyasapogenol-E-Monoglucosides'},
 'hmdb': {'HMDB0000122',
  'HMDB0000516',
  'HMDB0003340',
  'HMDB0003345',
  'HMDB0006564',
  'HMDB00122',
  'HMDB00516',
  'HMDB0061922',
  'HMDB0062170',
  'HMDB03340',
  'HMDB03345',
  'HMDB06564',
  'HMDB61922',
  'HMDB62170'},
 'bigg.metabolite': {'Glc_aD', 'glc_D_B', 'glc__D', 'glc__aD', 'glc__bD'},
 'seed.compound': {'cpd00027', 'cpd00190', 'cpd19001'},
 'chebi': {'10242',
  '10397',
  '12318',
  '12373',
  '15903',
  '17925',
  '22386',
  '22795',
  '40557',
  '41140',
  '4167',
  '42802',
  '88300'},
 'sabiork': {'1364', '1378', '1406', '1407'},
 'lipidmaps': set(),
 'envipath': {'32de3cf4-e3e6-4168-956e-32fa5ddb0ce1/compound/bee44ace-e680-4580-8d73-374e54ae70f8',
  '650babc9-9d68-4b73-9332-11972ca26f7b/compound/16de786